# 영산강 보 통합 데이터

**필수 라이브러리**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

**matplotlib 한글 설정**

- 운영체제에 따른 한글 지원 설정. 윈도우, 우분투, 구글 코랩 지원.
- 참고: [matplotlib에서 한글 지원하기](https://github.com/codingalzi/datapy/blob/master/matplotlib-korean.md)

In [2]:
import platform

if platform.system() == 'Windows': # 윈도우
    from matplotlib import font_manager, rc
    font_path = "C:/Windows/Fonts/NGULIM.TTF"
    font = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font)
elif platform.system() == 'Linux': # 우분투 또는 구글 코랩
    # please run the following commented out codes just once
#     if 'google.colab' in str(get_ipython()):
#         !apt-get install -y fonts-nanum*
#     else:
#         !sudo apt-get install -y fonts-nanum*
#     !fc-cache -fv
    
    applyfont = "NanumBarunGothic"
    import matplotlib.font_manager as fm
    if not any(map(lambda ft: ft.name == applyfont, fm.fontManager.ttflist)):
        fm.fontManager.addfont("/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf")
    plt.rc("font", family=applyfont)
    plt.rc("axes", unicode_minus=False)
    

## 데이터 준비

**데이터 저장소**

데이터 원본 파일 저장소는 다음과 같다.

In [3]:
base_url = "https://github.com/codingalzi/water-data/raw/master/reservoirs/"

**영산강(엑셀) 자료를 데이터프레임으로 불러오기**

모든 지역의 데이터 불러오기: 평동천, 광산, 장성천2, 문평천, 영산포2, 함평, 무안2

- `header=0`: 0번 행을 header로 지정, 즉 열 인덱스로 사용.
- `sheet_name=None`: 모든 워크시트 가져오기. 워크시트별로 하나의 df 생성. 반환값은 사전.
- `na_values=0`: 0으로 입력된 값도 결측치로 처리
- `index_col=1`: 측정일을 행 인덱스로 사용
- `parse_dates=True`: 행 인덱스로 사용되는 날짜 대상 파싱 실행

In [4]:
# 주의: 엑셀파일을 불러오기 위해 아래 모듈이 필요하다.

# !pip install openpyxl

In [5]:
yeongsan = pd.read_excel(base_url+"Yeongsan.xlsx",
                            header=0, 
                            sheet_name=None,
                            na_values=0,
                            index_col=1, 
                            parse_dates=True)

포함된 보(reservoir)의 지역명은 다음과 같다.

In [6]:
locations = yeongsan.keys()
locations

dict_keys(['1_평동천', '2_광산', '3_장성천2', '4_문평천', '5_영산포2', '6_함평', '7_무안2'])

**데이터 합치기**

지역별 데이터셋 크기는 다음과 같다.

In [7]:
total_data = 0

for loc in locations:
    ys_loc = yeongsan[loc]
    total_data += ys_loc.shape[0]
    print(f"{loc}: \t{ys_loc.shape}")
    
print("총 데이터수:", total_data)

1_평동천: 	(440, 19)
2_광산: 	(510, 19)
3_장성천2: 	(435, 19)
4_문평천: 	(435, 19)
5_영산포2: 	(456, 19)
6_함평: 	(456, 19)
7_무안2: 	(534, 19)
총 데이터수: 3266


모든 지역의 데이터를 단순히 합쳐서
총 3266개의 데이터 샘플을 갖는 데이터셋으로 만든다.

In [8]:
yongsan_total = pd.concat([yeongsan[loc] for loc in locations])
yongsan_total.shape

(3266, 19)

동일 날짜를 제외하면 1310개의 데이터에 불과하다.
하지만 장소가 다르기에 중복 날짜를 모두 인정한다. 

In [9]:
# 중복 날짜 수

yongsan_total.index.unique().shape

(1310,)

## 데이터 전처리

**주요 특성**

수온, BOD, COD, TN, TP, 유량 등 6개의 주요 특성만을 이용하여 클로로필-A 예측하려 한다.
원 데이터셋에 포함된 19개의 특성은 다음과 같다.

In [10]:
list(yongsan_total.columns)

['측정소명',
 '회차',
 '수온(℃)',
 'DO(㎎/L)',
 'BOD(㎎/L)',
 'COD(㎎/L)',
 '클로로필 a(㎎/㎥)',
 'TN(㎎/L)',
 'TP(㎎/L)',
 'TOC(㎎/L)',
 '수소이온농도',
 '전기전도도(μS/㎝)',
 '용존총질소(㎎/L)',
 '암모니아성 질소(㎎/L)',
 '질산성 질소(㎎/L)',
 '용존총인(㎎/L)',
 '인산염인(㎎/L)',
 'SS(㎎/L)',
 '유량(㎥/s)']

주요 특성 6개와 타깃으로 사용될 특성인 클로로필-A를 별도로 지정한다.

In [11]:
features_important = ['수온(℃)', 'BOD(㎎/L)', 'COD(㎎/L)', 'TN(㎎/L)', 'TP(㎎/L)', '유량(㎥/s)', '클로로필 a(㎎/㎥)']

- 입력데이터셋 특성 6개

In [12]:
six_features = features_important[:6]

* 타깃 특성: 클로로필-A

In [13]:
target_feature = features_important[-1]

**타깃 결측치 제거**

타깃으로 사용될 클로로필-A 특성에 48개의 결측치가 존재한다.

In [14]:
yongsan_total[target_feature].isna().sum()

48

타깃에 결측치가 포함되면 안되기에 해당 데이터 샘플들을 제거한 후에
총 3218개의 데이터만 사용한다.

In [15]:
mask = yongsan_total[target_feature].isna()
yongsan = yongsan_total[~mask]

yongsan.shape

(3218, 19)

**핵심 특성만 사용**

이제부터 언급된 7개의 특성만 사용한다.

In [16]:
ys_seven = yongsan[features_important]

ys_seven

,수온(℃),BOD(㎎/L),COD(㎎/L),TN(㎎/L),TP(㎎/L),유량(㎥/s),클로로필 a(㎎/㎥)
년/월/일,,,,,,,
2012-09-04,24.0,2.0,11.0,2.651,0.123,0.677,20.4
2012-09-11,24.5,1.8,9.5,2.433,0.107,0.892,7.2
2012-09-18,20.8,2.7,9.0,2.042,0.132,3.393,25.2
2012-09-25,20.6,1.4,5.6,2.187,0.060,0.585,12.0
2012-11-08,11.8,1.8,5.7,3.393,0.074,0.242,9.2
...,...,...,...,...,...,...,...
2022-05-31,22.7,2.9,7.7,3.598,0.043,141.215,5.9
2022-06-08,22.8,3.0,8.0,3.254,0.050,287.252,18.8
2022-06-13,24.0,2.6,8.6,2.843,0.044,NaN,17.7


**입력 데이터셋 결측치 처리**

유량 특성에 여전히 335개 결측치가 포함되어 있다.

In [17]:
ys_seven.isna().sum()

수온(℃)            0
BOD(㎎/L)         0
COD(㎎/L)         0
TN(㎎/L)          0
TP(㎎/L)          0
유량(㎥/s)        335
클로로필 a(㎎/㎥)      0
dtype: int64

335개는 전체 데이터셋의 10% 정도이다.

In [18]:
num_missing_values = ys_seven.isna().sum().sum()
num_missing_values/ys_seven.shape[0]

0.10410192666252331

유량 특성에만 있는 결측치는 모두 0으로 대체한다.

RNN 모델에 드롭아웃(dropout)을 적용하면 일부 특성이 어차피 0으로 지정된다.
따라서 입력 데이터셋의 결측치가 너무 많지 않다면
드롭아웃 효과에 의해 결측치의 영향이 묻히게 된다.

In [19]:
ys_seven = ys_seven.fillna(0)
ys_seven.isna().sum()

수온(℃)          0
BOD(㎎/L)       0
COD(㎎/L)       0
TN(㎎/L)        0
TP(㎎/L)        0
유량(㎥/s)        0
클로로필 a(㎎/㎥)    0
dtype: int64

**특성 정규화**

모든 특성을 정규화한다. 

먼저 특성별 평균값과 표준편차를 계산한다.

In [20]:
ys_mean = ys_seven.mean(axis=0)
ys_std = ys_seven.std(axis=0)

평균은 0, 표준편차는 1로 변환한다.

In [21]:
ys_seven = (ys_seven - ys_mean)/ys_std
ys_seven

,수온(℃),BOD(㎎/L),COD(㎎/L),TN(㎎/L),TP(㎎/L),유량(㎥/s),클로로필 a(㎎/㎥)
년/월/일,,,,,,,
2012-09-04,0.934475,-0.841111,1.229895,-0.738148,-0.191344,-0.218735,-0.212939
2012-09-11,0.996661,-0.949860,0.668484,-0.859148,-0.340261,-0.218198,-0.647142
2012-09-18,0.536479,-0.460491,0.481346,-1.076170,-0.107578,-0.211948,-0.055047
2012-09-25,0.511605,-1.167357,-0.791187,-0.995688,-0.777704,-0.218965,-0.489250
2012-11-08,-0.582882,-0.949860,-0.753760,-0.326306,-0.647402,-0.219822,-0.581354
...,...,...,...,...,...,...,...
2022-05-31,0.772789,-0.351743,-0.005211,-0.212523,-0.935928,0.132430,-0.689904
2022-06-08,0.785226,-0.297369,0.107072,-0.403457,-0.870777,0.497334,-0.265570
2022-06-13,0.934475,-0.514866,0.331637,-0.631580,-0.926621,-0.220426,-0.301753


## 시계열 데이터 분석

시간의 흐름을 고려해서 시계열(timeseries) 데이터로 처리한다.

**날짜별로 정렬**

먼저 날짜별로 정렬한다.

In [22]:
ys_seven = ys_seven.sort_index()

**훈련셋과 테스트셋 지정**

훈련셋, 검증셋, 테스트셋을 7:2:1의 비율로 나눈다.
단, 테스트셋은 날짜를 기준으로 나중에 측정된 데이터를 이용한다.

In [23]:
train_size = int(ys_seven.shape[0] * 0.7)
val_size = int(ys_seven.shape[0] * 0.2)

- 훈련셋

In [24]:
train_set = ys_seven[six_features][:train_size]
train_targets = ys_seven[target_feature][:train_size]

- 검증셋

In [25]:
val_set = ys_seven[six_features][train_size : train_size + val_size]
val_targets = ys_seven[target_feature][train_size : train_size + val_size]

- 테스트셋

In [26]:
test_set = ys_seven[six_features][train_size + val_size:]
test_targets = ys_seven[target_feature][train_size + val_size:]

**시계열 데이터로 변환**

In [27]:
import tensorflow as tf

시계열 데이터 샘플을 `sequence_length` 만큼의 타임 스텝(time step)로
구성한다.
예측값은 미래가 아닌 현재의 클로로필-A 수치로 지정한다.

- 배치 크기: 32
- 공정한 훈련을 위해 구성된 시계열 샘플을 뒤섞는다.

In [28]:
sequence_length=50  # 타임 스텝 크기

train_dataset = tf.keras.utils.timeseries_dataset_from_array(
    train_set,
    targets=train_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=32)

val_dataset = tf.keras.utils.timeseries_dataset_from_array(
    val_set,
    targets=val_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=32)

test_dataset = tf.keras.utils.timeseries_dataset_from_array(
    test_set,
    targets=test_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=32)

입력 데이터셋의 배치의 모양은 `(32, 50, 6)`이다.

In [29]:
for samples, targets in train_dataset:
    print("samples shape:", samples.shape)
    print("targets shape:", targets.shape)
    break

samples shape: (32, 50, 6)
targets shape: (32,)


**케라스 활용**

In [31]:
from tensorflow import keras
from tensorflow.keras import layers

**`Dense` 모델 사용**

타임시리즈로 구성된 샘플을 1차원 텐서로 푼 다음에 `Dense` 모델을 적용해본다.

In [39]:
inputs = keras.Input(shape=(sequence_length, len(features_important)-1))
x = layers.Flatten()(inputs)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dense(1024, activation="relu")(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("yeongsan_dense.keras",
                                    save_best_only=True)
] 

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

history = model.fit(train_dataset,
                    epochs=100,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("yeongsan_dense.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/100
69/69 [==============================] - 1s 8ms/step - loss: 4.1348 - mae: 0.9702 - val_loss: 1.2239 - val_mae: 0.6474
Epoch 2/100
69/69 [==============================] - 0s 5ms/step - loss: 0.6139 - mae: 0.5589 - val_loss: 0.9480 - val_mae: 0.6598
Epoch 3/100
69/69 [==============================] - 0s 4ms/step - loss: 0.3952 - mae: 0.4407 - val_loss: 1.3256 - val_mae: 0.8075
Epoch 4/100
69/69 [==============================] - 0s 5ms/step - loss: 0.2905 - mae: 0.3814 - val_loss: 0.8903 - val_mae: 0.6489
Epoch 5/100
69/69 [==============================] - 0s 4ms/step - loss: 0.2302 - mae: 0.3362 - val_loss: 0.9979 - val_mae: 0.5602
Epoch 6/100
69/69 [==============================] - 0s 5ms/step - loss: 0.1812 - mae: 0.3046 - val_loss: 0.8568 - val_mae: 0.5155
Epoch 7/100
69/69 [==============================] - 0s 4ms/step - loss: 0.1381 - mae: 0.2616 - val_loss: 0.8840 - val_mae: 0.5307
Epoch 8/100
69/69 [==============================] - 0s 4ms/step - loss: 0.1363 - m

69/69 [==============================] - 0s 4ms/step - loss: 0.0140 - mae: 0.0745 - val_loss: 0.7118 - val_mae: 0.4960
Epoch 64/100
69/69 [==============================] - 0s 4ms/step - loss: 0.0163 - mae: 0.0767 - val_loss: 0.7425 - val_mae: 0.4791
Epoch 65/100
69/69 [==============================] - 0s 4ms/step - loss: 0.0127 - mae: 0.0727 - val_loss: 0.7165 - val_mae: 0.4835
Epoch 66/100
69/69 [==============================] - 0s 4ms/step - loss: 0.0161 - mae: 0.0765 - val_loss: 0.7240 - val_mae: 0.4782
Epoch 67/100
69/69 [==============================] - 0s 4ms/step - loss: 0.0128 - mae: 0.0712 - val_loss: 0.7559 - val_mae: 0.4962
Epoch 68/100
69/69 [==============================] - 0s 4ms/step - loss: 0.0138 - mae: 0.0733 - val_loss: 0.7452 - val_mae: 0.4887
Epoch 69/100
69/69 [==============================] - 0s 4ms/step - loss: 0.0132 - mae: 0.0714 - val_loss: 0.7241 - val_mae: 0.4907
Epoch 70/100
69/69 [==============================] - 0s 4ms/step - loss: 0.0140 - mae: 0

**GRU 모델 사용**

In [80]:
inputs = keras.Input(shape=(sequence_length, len(features_important)-1))
x = layers.GRU(50, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(50, recurrent_dropout=0.5, return_sequences=True)(x)
x = layers.GRU(50, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
  
callbacks = [
    keras.callbacks.ModelCheckpoint("yeongsan_gru.keras",
                                    save_best_only=True)
]
                     
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("yeongsan_gru.keras") 

print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/10
69/69 [==============================] - 34s 433ms/step - loss: 0.8217 - mae: 0.6627 - val_loss: 1.1413 - val_mae: 0.6798
Epoch 2/10
69/69 [==============================] - 28s 413ms/step - loss: 0.6343 - mae: 0.5688 - val_loss: 0.7891 - val_mae: 0.5190
Epoch 3/10
69/69 [==============================] - 29s 419ms/step - loss: 0.4731 - mae: 0.4842 - val_loss: 0.6919 - val_mae: 0.4906
Epoch 4/10
69/69 [==============================] - 29s 419ms/step - loss: 0.4533 - mae: 0.4660 - val_loss: 0.6752 - val_mae: 0.4656
Epoch 5/10
69/69 [==============================] - 29s 420ms/step - loss: 0.4289 - mae: 0.4531 - val_loss: 0.6503 - val_mae: 0.4662
Epoch 6/10
69/69 [==============================] - 28s 408ms/step - loss: 0.4021 - mae: 0.4398 - val_loss: 0.6435 - val_mae: 0.4886
Epoch 7/10
69/69 [==============================] - 28s 408ms/step - loss: 0.3999 - mae: 0.4426 - val_loss: 0.6094 - val_mae: 0.4515
Epoch 8/10
69/69 [==============================] - 29s 412ms/step - 

In [87]:
inputs = keras.Input(shape=(sequence_length, len(features_important)-1))
x = layers.GRU(32, return_sequences=True)(inputs)
x = layers.GRU(32, return_sequences=True)(x)
x = layers.GRU(32, return_sequences=True)(x)
x = layers.GRU(32)(x)
# x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
  
early_stopping_cb = keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=10, restore_best_weights=True)

callbacks = [
    keras.callbacks.ModelCheckpoint("yeongsan_gru.keras",
                                    save_best_only=True),
    early_stopping_cb
]
                     
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

history = model.fit(train_dataset,
                    epochs=100,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("yeongsan_gru.keras") 

print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/100
69/69 [==============================] - 6s 31ms/step - loss: 0.8490 - mae: 0.6776 - val_loss: 1.2436 - val_mae: 0.7500
Epoch 2/100
69/69 [==============================] - 1s 19ms/step - loss: 0.6097 - mae: 0.5468 - val_loss: 0.7129 - val_mae: 0.4851
Epoch 3/100
69/69 [==============================] - 1s 18ms/step - loss: 0.4035 - mae: 0.4298 - val_loss: 0.6162 - val_mae: 0.4537
Epoch 4/100
69/69 [==============================] - 1s 18ms/step - loss: 0.3787 - mae: 0.4113 - val_loss: 0.5900 - val_mae: 0.4466
Epoch 5/100
69/69 [==============================] - 1s 18ms/step - loss: 0.3580 - mae: 0.3968 - val_loss: 0.5905 - val_mae: 0.4648
Epoch 6/100
69/69 [==============================] - 1s 19ms/step - loss: 0.3455 - mae: 0.3881 - val_loss: 0.5881 - val_mae: 0.4569
Epoch 7/100
69/69 [==============================] - 1s 18ms/step - loss: 0.3338 - mae: 0.3838 - val_loss: 0.6138 - val_mae: 0.4921
Epoch 8/100
69/69 [==============================] - 1s 18ms/step - loss: 0.

**모델 지정**